In [5]:
import numpy as np

def local_thresholding(img, window_size):
    half_window = window_size // 2
    top_rows = img[:half_window, :][::-1, :]
    bottom_rows = img[-half_window:, :][::-1, :]
    #Matrix zusammenfügen
    imgM = np.concatenate((top_rows, img, bottom_rows), axis=0)
    
    # Äußere Spalten spiegeln und erweitern
    left_cols = imgM[:, :half_window][:, ::-1]
    right_cols = imgM[:, -half_window:][:, ::-1]
    imgM = np.concatenate((left_cols, imgM, right_cols), axis=1)
    imgT = imgM.copy()
    height, width = imgM.shape[:2]

    for y in range(half_window, height - half_window):
        for x in range(half_window, width - half_window):
            window = imgM[y - half_window : y + half_window + 1, x - half_window : x + half_window + 1]
             
            rvl = window.ravel()
            ran = round(max(rvl) - min(rvl))
            counts, bins = np.histogram(rvl, bins=ran)
 
            variance_list = []
 
            for T in range(1, len(counts)):
                sum_back = 0 
                mean_sum_back = 0
                for i in range(0, T):
                    sum_back += counts[i]
                    mean_sum_back += counts[i] * bins[i]
 
                w0 = sum_back / sum(counts)
                mean_back = mean_sum_back / sum_back
 
                sum_obj = 0
                mean_sum_obj = 0
                for j in range(T, len(counts)):
                    sum_obj += counts[j]
                    mean_sum_obj += counts[j] * bins[j]
 
                w1 = sum_obj / sum(counts)
                mean_obj = mean_sum_obj / sum_obj
 
                var_sum_back = 0
                for k in range(0, T):
                    var_sum_back += counts[k] * (bins[k] - mean_back)**2 
 
                var_back = var_sum_back / sum_back
 
                var_sum_obj = 0
                for l in range(T, len(counts)):
                    var_sum_obj += counts[l] * (bins[l] - mean_obj)**2 
 
                var_obj = var_sum_obj / sum_obj
 
                within_class_varianz = w0 * var_back + w1 * var_obj
                variance_list.append(within_class_varianz)
 
            minvar = min(variance_list)
            THRESH = round(bins[variance_list.index(minvar)])
 
            if imgM[y, x] < THRESH:
                imgT[y, x] = 0
            else:
                imgT[y, x] = 255

    return imgT

In [16]:
#local thresholding Vektorisiert

import numpy as np

def local_thresholding_vek(img, window_size):
    half_window = window_size // 2
    top_rows = img[:half_window, :][::-1, :]
    bottom_rows = img[-half_window:, :][::-1, :]
    imgM = np.concatenate((top_rows, img, bottom_rows), axis=0)
    
    left_cols = imgM[:, :half_window][:, ::-1]
    right_cols = imgM[:, -half_window:][:, ::-1]
    imgM = np.concatenate((left_cols, imgM, right_cols), axis=1)
    imgT = imgM.copy()
    height, width = imgM.shape[:2]

    for y in range(half_window, height - half_window):
        for x in range(half_window, width - half_window):
            window = imgM[y - half_window : y + half_window + 1, x - half_window : x + half_window + 1]
             
            rvl = window.ravel()
            ran = round(max(rvl) - min(rvl))
            counts, bins = np.histogram(rvl, bins=ran)
            
            cumulative_sum = np.cumsum(counts)
            cumulative_mean = np.cumsum(bins[:-1] * counts) / cumulative_sum
            
            within_class_variance = (cumulative_mean[cumulative_sum - cumulative_sum[-1]] - cumulative_mean) ** 2 / cumulative_sum / (1 - cumulative_sum)
            minvar = np.min(within_class_variance)
            THRESH = round(bins[:-1][within_class_variance == minvar][0])
 
            if imgM[y, x] < THRESH:
                imgT[y, x] = 0
            else:
                imgT[y, x] = 255

    return imgT


In [17]:
#Anwendung auf  3 Datensatz
from skimage import io
img1 = io.imread('../BilderDaten/BilderDaten/NIH3T3/img/dna-1.png')
print(img1.shape)
io.imshow(local_thresholding_vek(io.imread('../BilderDaten/BilderDaten/NIH3T3/img/dna-1.png'),window_size=100))

(1024, 1344)


IndexError: index -10189 is out of bounds for axis 0 with size 4

In [13]:
#Versuch schneller local
import numpy as np

def local_thresholding_vek2(img, window_size):
    imgT=img.copy()
    half_window = window_size // 2
  
    height, width = img.shape[:2]

    for y in range(half_window, height - half_window):
        for x in range(half_window, width - half_window):
            window = img[y - half_window : y + half_window + 1, x - half_window : x + half_window + 1]
             
            rvl = window.ravel()
            ran = round(max(rvl) - min(rvl))
            counts, bins = np.histogram(rvl, bins=ran)
            
            cumulative_sum = np.cumsum(counts)
            cumulative_mean = np.cumsum(bins[:-1] * counts) / cumulative_sum
            
            within_class_variance = (cumulative_mean[cumulative_sum - cumulative_sum[-1]] - cumulative_mean) ** 2 / cumulative_sum / (1 - cumulative_sum)
            minvar = np.min(within_class_variance)
            THRESH = round(bins[:-1][within_class_variance == minvar][0])
 
            if img[y, x] < THRESH:
                imgT[y, x] = 0
            else:
                imgT[y, x] = 255

    return imgT

In [14]:
#Anwendung auf  3 Datensatz
from skimage import io
io.imshow(local_thresholding_vek2(io.imread('../BilderDaten/BilderDaten/NIH3T3/img/dna-1.png'),window_size=100))

IndexError: index -10178 is out of bounds for axis 0 with size 4

In [3]:
#Versucherweiterung Matrix

import numpy as np

def spiegel_matrix(matrix):
    rows, cols = matrix.shape

    # Äußere Zeilen spiegeln und erweitern
    top_rows = matrix[:3, :][::-1, :]
    bottom_rows = matrix[-3:, :][::-1, :]
    #Matrix zusammenfügen
    matrix = np.concatenate((top_rows, matrix, bottom_rows), axis=0)
    
    print(top_rows)
    print(bottom_rows)
    

    # Äußere Spalten spiegeln und erweitern
    left_cols = matrix[:, :3][:, ::-1]
    right_cols = matrix[:, -3:][:, ::-1]
    matrix = np.concatenate((left_cols, matrix, right_cols), axis=1)

    return matrix

# Beispielmatrix
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Matrix spiegeln und erweitern
#neue_matrix = spiegel_matrix(matrix)

print(neue_matrix)


[[1 2 3 1 2 3]
 [4 5 6 4 5 6]
 [7 8 9 7 8 9]]
